In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Skola/Kryptografia/md5/'

with open(path + 'mena.txt') as f:
    lines = f.readlines()


Mounted at /content/drive


Vygenerovanie mien

In [ ]:
import unicodedata

names = []
for l in lines:
  l = l.replace('\n', '')
  l = l.lower()
  l = unicodedata.normalize('NFKD', l).encode('ascii', 'ignore').decode('utf-8')

  names.append(l)

  for c in range(len(l)):
    names.append(f'{l[:c]}{l[c].upper()}{l[c + 1:]}')

  if(l[-1] == 'a'):
    if(l[-2]) == 'i':
      names.append(l[:-1] + 'nka')
    elif(l[-2] == 'n'):
      names.append(l[:-1] + 'ka')
    elif(l[-2] != 'k'):
      names.append(l[:-1] + 'ka')
  else:
    names.append(l + 'ko')

  l = names[-1]
  for c in range(len(l)):
    names.append(f'{l[:c]}{l[c].upper()}{l[c + 1:]}')

Prihlasovacie údaje

In [ ]:
data = []
for i in range(4):
  with open(path + f'shadow{i + 1}.txt') as f:
      lines = f.readlines()
  
  for l in range(len(lines)):
    lines[l] = lines[l].replace('\n', '').split(':')
  
  data.append(lines)

In [ ]:
from multiprocessing import Process, Lock
from hashlib import md5
from base64 import b64encode

def f(l, id, n, d):
    for psw in n:
      for rd in d:
        m = md5()
        m.update( psw.encode('utf-8') )
        m.update( rd[1].encode('utf-8') ) # salt
        hash = b64encode( m.digest())
        if(hash == rd[2].encode('utf-8')):
          l.acquire()
          try:
            print(f'{id} > {rd[0]} has password : {psw}')
          finally:
            l.release()

if __name__ == '__main__':
    lock = Lock()
    workers = []
    for id in range(4):
        worker = Process(target=f, args=(lock, id + 1, names, data[id]))
        worker.start()
        workers.append(worker)

    for worker in workers:
      worker.join()


3 > kurka has password : adamKo
4 > slovakova5 has password : adamKo
2 > sagan00 has password : kAtarina
1 > scensna has password : maTusko
2 > pachnik has password : maTusko
2 > potkan3 has password : maTusko
3 > potkan3 has password : maTusko
3 > kysela9 has password : milaDa


In [ ]:
pip install hashcat

     |████████████████████████████████| 95.7 MB 58 kB/s 


In [ ]:
from base64 import b64decode
for d in data:
  for dd in d:
    print(b64decode(dd[2]).hex() + ':' + dd[1])

hash mode : 10	md5($pass.$salt)	01dfae6e5d4d90d9892622325959afbe:7050461
bruteforce
mask 1 (6-7) l *chars*
mask 2 (4-5) l/u/d chars
-1?l ?1?1?1?1?1?1?1 --increment --increment-min 6

In [ ]:
!hashcat -a 3 /content/drive/MyDrive/Skola/Kryptografia/md5/hashes.txt -m 10 -o /content/drive/MyDrive/Skola/Kryptografia/md5/cracked.txt  -2?l?u?d ?2?2?2?2?2 --increment --increment-min 4

hashcat (v6.2.5) starting

* Device #1: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
* Device #2: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 11.2)
* Device #1: Tesla K80, 11382/11441 MB, 13MCU

OpenCL API (OpenCL 1.2 CUDA 11.2.109) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla K80, skipped

Minimum password length supported by kernel: 0
Maximum password length supported by kernel: 256
Minimim salt length supported by kernel: 0
Maximum salt length supported by kernel: 256

Hashes: 400 digests; 400 unique digests, 400 unique salts
Bitmaps: 16 bits, 65536 entries, 0x0000ffff mask, 262144 bytes, 5/13 rotates

